In [2]:
import pyodbc

In [3]:
## Hotel
class Hotel:
    menu = {}
    conn = pyodbc.connect(
        "Driver={SQL Server};"
        "Server=DAVIDG\\SQLEXPRESS01;"
        "Database=Hotel_db;"
        "Trusted_Connection=yes"
    )

    def __init__(self):
        print("Welcome to the Hotel Spice")
        self.cursor = self.conn.cursor()
        self.cursor.execute("Select * from dbo.MENU")

    def Show_menu(self):
        records = self.cursor.fetchall()
        for row in records:
            self.menu[row.Menu_id] = {
                "name": row.ItemName,
                "half": row.Half_price,
                "full": row.Full_price,
            }
        print("--Menu--")
        for Mid, item in self.menu.items():
            print(
                f"{Mid}. {item['name']} | Half: ₹{item['half']} | Full: ₹{item['full']}"
            )

    def captain(self):
        print("What would you like to have: ")

In [ ]:
class Order(Hotel):
    global_order_id = 0

    def __init__(self):
        super().__init__()

        Order.global_order_id += 1
        self.order_id = Order.global_order_id

        self.captain()

    def take_order(self):
        while True:
            try:

                ItemName = input("Enter item: ")
                Portion = input("Enter the item type (Half/Full): ").capitalize()
                try:
                    QTY = int(input("Enter the Quantity: "))
                    if QTY <=0:
                        raise ValueError("Quantity must be greater than zero")
                except ValueError:
                    print("Invalid Quantity, Please enter correct value.")
                    continue

                # Fetch price from MENU
                self.cursor.execute(
                     "SELECT Half_price, Full_price FROM dbo.MENU WHERE ItemName = ?",
                    (ItemName,),
                )
                price_data = self.cursor.fetchone()

                if not price_data:
                    print(" Item not found in menu!")
                    continue

                if Portion == "Half":
                    price = price_data[0]
                elif Portion == "Full":
                    price = price_data[1]
                else:
                    print(" Invalid portion! Choose Half or Full.")
                    continue

                total = price * QTY

                # Insert in order table
                self.cursor.execute(
                    """
                    INSERT INTO Orders 
                    (OrderID, ItemName, Portion, Quantity, Price, TotalAmount)
                    VALUES (?, ?, ?, ?, ?, ?)
                """,
                    (self.order_id, ItemName, Portion, QTY, price, total),
                )

                self.conn.commit()

                print(" Item added successfully to database")

                more = input("Is it your final order? (Y for Yes | N for No): ").upper()
                if more == "Y":
                    break
            except Exception as e:
                print(f"Something went wrong: {e}")
                print("Transaction rolled back.")
                self.conn.rollback()

In [9]:
class Bill(Order):
    def __init__(self, order_id):
        super().__init__()
        self.order_id = order_id

    def generate_bill(self):
        self.cursor.execute(
            """
            SELECT ItemName, Portion, Quantity, Price, TotalAmount, OrderDate
            FROM Orders
            WHERE OrderID = ?
        """,
            (self.order_id,),
        )

        records = self.cursor.fetchall()

        if not records:
            print("No records found")
            return

        # Take date from first row
        order_date = records[0][5]

        print("\n" + "=" * 50)
        print("           HOTEL SPICE BILL")
        print("=" * 50)
        print(f"Order ID   : {self.order_id}")
        print(f"Order Date : {order_date}")
        print("-" * 50)
        print(f"{'Item':15}{'Portion':10}{'Qty':5}{'Price':10}{'Total':10}")
        print("-" * 50)

        grand_total = 0

        for row in records:
            item, portion, qty, price, total, _ = row
            price = int(price)
            total = int(total)
            grand_total += total

            print(f"{item:15}{portion:10}{qty:<5}{price:<10}{total:<10}")

        tax = (grand_total * 5) // 100
        final = grand_total + tax

        print("-" * 50)
        print(f"Subtotal     : ₹{grand_total}")
        print(f"GST 5%       : ₹{tax}")
        print(f"Grand Total  : ₹{final}")
        print("=" * 50)

In [10]:
user_2 = Order()
user_2.Show_menu()
user_2.take_order()
user_2_bill = Bill(Order.global_order_id)
user_2_bill.generate_bill()

Welcome to the Hotel Spice
What would you like to have: 
--Menu--
1. Paneer Butter Masala | Half: ₹180 | Full: ₹320
2. Paneer Tikka Masala | Half: ₹190 | Full: ₹340
3. Shahi Paneer | Half: ₹200 | Full: ₹360
4. Kadai Paneer | Half: ₹190 | Full: ₹330
5. Dal Tadka | Half: ₹120 | Full: ₹220
6. Dal Fry | Half: ₹110 | Full: ₹210
7. Veg Kolhapuri | Half: ₹170 | Full: ₹300
8. Mix Veg | Half: ₹160 | Full: ₹290
9. Aloo Gobi | Half: ₹140 | Full: ₹260
10. Butter Naan | Half: ₹40 | Full: ₹70
11. Plain Roti | Half: ₹20 | Full: ₹35
12. Jeera Rice | Half: ₹110 | Full: ₹200
13. Veg Fried Rice | Half: ₹140 | Full: ₹260
14. Veg Biryani | Half: ₹160 | Full: ₹300
15. Chicken Curry | Half: ₹200 | Full: ₹360
16. Chicken Butter Masala | Half: ₹220 | Full: ₹390
17. Chicken Biryani | Half: ₹230 | Full: ₹420
18. Egg Curry | Half: ₹150 | Full: ₹280
19. Masala Omelette | Half: ₹90 | Full: ₹160
20. Gulab Jamun | Half: ₹60 | Full: ₹110
21. Tandoori Roti | Half: ₹25 | Full: ₹45
22. Butter Roti | Half: ₹30 | Full: ₹55

25